In [173]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint
from statsmodels.sandbox.stats.multicomp import multipletests 
import multiprocessing as mp

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [62]:
gene_data = pd.read_csv("gene_high_throughput_sequencing.csv", sep=",") 

In [280]:
gene_cols = gene_data.columns[2:]
gene_data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [281]:
gene_data_normal = gene_data[gene_data['Diagnosis'] == 'normal']
gene_data_early = gene_data[gene_data['Diagnosis'] == 'early neoplasia']
gene_data_cancer = gene_data[gene_data['Diagnosis'] == 'cancer']

In [313]:
# define practically valuable metric: fold change
def fold_change(C, T): 
    if(T > C):
        return T / C
    else:
        return  C / T

# calculate t-statistic and p-value for genes
def stat_pract_significance(args): 
    control, treatment, genes_names = args
    data = pd.DataFrame()
    data['gene_name'] = np.array(genes_names)
    data = data.set_index(['gene_name'])
    
    data['mean_control'] = np.zeros(len(genes_names))
    data['mean_treatment'] = np.zeros(len(genes_names))
    data['F_c'] = np.zeros(len(genes_names))
    
    data['t_stat'] = np.zeros(len(genes_names))
    data['p_value'] = np.zeros(len(genes_names))
    data['h0_rejected'] = numpy.full(len(genes_names), numpy.nan)
    data['F_c_change_significant'] = numpy.full(len(genes_names), numpy.nan)
    
    for gene in genes_names: 
        t_test_result = scipy.stats.ttest_ind(control[gene], treatment[gene], equal_var = False)
        
        n_mean = control[gene].mean()
        d_mean = treatment[gene].mean()
        F_c = fold_change(n_mean, d_mean)
        t_stat = t_test_result.statistic
        p_value = t_test_result.pvalue
        
        data.loc[gene,[ 'mean_control', 'mean_treatment', 'F_c', 't_stat', 'p_value']] = [n_mean, d_mean, F_c, t_stat, p_value]
        data.loc[gene,['h0_rejected']] = p_value < 0.05 # confidence level statistical
        data.loc[gene,['F_c_change_significant']] = np.abs(F_c) > 1.5 # confidence level practical 

    return data

# multiple hypothesis testing
#Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, 
#то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value 
#c помощью метода Холма.
def stat_pract_significance_multiple(stat, alpha = 0.025, method = 'holm'):
    reject, p_corrected, a1, a2 = multipletests(stat['p_value'], alpha, method) 
    
    
    
    stat['p_corrected_' + method] = p_corrected
    stat['h0_rejected_' + method] = reject
#     stat['h0_rejected_' + method] = p_corrected < 0.05 # confidence level is the same
    
    return stat

#calculations, Part 1
def part1(control, treatment, gene_cols, name): 
    data = stat_pract_significance((control, treatment, gene_cols))
    res_stat = data[data['h0_rejected'] == True].h0_rejected.count()

    print('Amount of genes with statistical difference for ['+name+'] :', res_stat)
    return data
    
def part23(res_stat, name, alpha = 0.025, method = 'holm'):
    stat_pract_significance_multiple(res_stat, alpha, method)
    
    rejected = res_stat[res_stat['h0_rejected_'+method] == True]
    
    rejected_stat = rejected['h0_rejected_'+method].count()
    
    rejected_practical = res_stat[res_stat['F_c_change_significant'] == True]['F_c_change_significant'].count()
    
    rejected_practical_stat = rejected[rejected['F_c_change_significant'] == True]['F_c_change_significant'].count()
    
    print('Amount of genes with statistical difference using ['+method+'] for ['+name+'] :', rejected_stat)
    print('Amount of genes with practical difference using ['+method+'] for ['+name+'] :', rejected_practical)
    print('Amount of genes with practical & statistical difference using ['+method+'] for ['+name+'] :', rejected_practical_stat)


#https://stackoverflow.com/questions/41385708/multiprocessing-example-giving-attributeerror
#cpu_count = mp.cpu_count()
#pool = mp.Pool(processes=cpu_count)
# dfs = pool.map( statistical_significance, [(gene_data_normal, gene_data_early, genes_col.array) for genes_col in numpy.array_split(gene_cols, cpu_count)])
# data_all_new = pd.concat( dfs )

# PART 1

In [299]:
en_stat = part1(gene_data_normal, gene_data_early, gene_cols, 'early neoplasia')

Amount of genes with statistical difference for [early neoplasia] : 1575


In [300]:
cancer_stat = part1(gene_data_normal, gene_data_cancer, gene_cols, 'cancer')

Amount of genes with statistical difference for [cancer] : 4059


# PART 2

In [318]:
part23(en_stat, name = 'early neoplasia', alpha = 0.025, method = 'holm')

Amount of genes with statistical difference using [holm] for [early neoplasia] : 2
Amount of genes with practical difference using [holm] for [early neoplasia] : 261
Amount of genes with practical & statistical difference using [holm] for [early neoplasia] : 2


In [319]:
part23(cancer_stat,name = 'cancer', alpha = 0.025, method = 'holm')

Amount of genes with statistical difference using [holm] for [cancer] : 163
Amount of genes with practical difference using [holm] for [cancer] : 1449
Amount of genes with practical & statistical difference using [holm] for [cancer] : 146


# PART 3

In [320]:
part23(en_stat,name = 'early neoplasia',alpha = 0.025, method = 'fdr_bh')

Amount of genes with statistical difference using [fdr_bh] for [early neoplasia] : 4
Amount of genes with practical difference using [fdr_bh] for [early neoplasia] : 261
Amount of genes with practical & statistical difference using [fdr_bh] for [early neoplasia] : 4


In [321]:
part23(cancer_stat,name = 'cancer',alpha = 0.025, method = 'fdr_bh')

Amount of genes with statistical difference using [fdr_bh] for [cancer] : 1419
Amount of genes with practical difference using [fdr_bh] for [cancer] : 1449
Amount of genes with practical & statistical difference using [fdr_bh] for [cancer] : 840


In [307]:
with open('answer1.txt', 'w') as fout:
    fout.write(str(1575))
    
with open('answer2.txt', 'w') as fout:
    fout.write(str(4059))

In [308]:
with open('answer3.txt', 'w') as fout:
    fout.write(str(2))
    
with open('answer4.txt', 'w') as fout:
    fout.write(str(146))

In [309]:
with open('answer5.txt', 'w') as fout:
    fout.write(str(4))
    
with open('answer6.txt', 'w') as fout:
    fout.write(str(840))